### Clustering locations into groups of similar coordinates

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth
# in-situ coordinates
insitu_coords = pd.read_csv('geometry.txt',sep=' ')

y = insitu_coords[['xcoord','ycoord','zcoord']].values
bw = estimate_bandwidth(y, quantile=.01)
ms = MeanShift(bandwidth=bw, bin_seeding=True, min_bin_freq=5)
ms.fit(y)
# store coordinates for the centers of the clusters
cluster_centers = ms.cluster_centers_
np.save('cluster_centers.npy',cluster_centers)

# store what cluster the insitu coordinates belong to
cluster_label = list(ms.labels_)
with open('cluster_label.txt','w') as f:
    f.write('\n'.join(map(str,cluster_label)))

### Feature selection

In [2]:
import numpy as np
import pandas as pd
import sys
import math
import sklearn.cluster
from skfeature.utility.construct_W import construct_W
from skfeature.utility.sparse_learning import feature_ranking

def ndfs(X, **kwargs):
    """
    This function implement unsupervised feature selection using nonnegative spectral analysis, i.e.,
    min_{F,W} Tr(F^T L F) + alpha*(||XW-F||_F^2 + beta*||W||_{2,1}) + gamma/2 * ||F^T F - I||_F^2
    s.t. F >= 0
    
    Input
    -----
    X: {numpy array}, shape (n_samples, n_features)
        input data
    kwargs: {dictionary}
        W: {sparse matrix}, shape {n_samples, n_samples}
            affinity matrix
        alpha: {float}
            Parameter alpha in objective function
        beta: {float}
            Parameter beta in objective function
        gamma: {float}
            a very large number used to force F^T F = I
        F0: {numpy array}, shape (n_samples, n_clusters)
            initialization of the pseudo label matirx F, if not provided
        n_clusters: {int}
            number of clusters
        verbose: {boolean}
            True if user want to print out the objective function value in each iteration, false if not

    Output
    ------
    W: {numpy array}, shape(n_features, n_clusters)
        feature weight matrix
        
    Reference: 
        Li, Zechao, et al. "Unsupervised Feature Selection Using Nonnegative Spectral Analysis." AAAI. 2012.
    """

    # default gamma is 10e8
    if 'gamma' not in kwargs:
        gamma = 10e8
    else:
        gamma = kwargs['gamma']
    # use the default affinity matrix
    if 'W' not in kwargs:
        W = construct_W(X)
    else:
        W = kwargs['W']
    if 'alpha' not in kwargs:
        alpha = 1
    else:
        alpha = kwargs['alpha']
    if 'beta' not in kwargs:
        beta = 1
    else:
        beta = kwargs['beta']
    if 'F0' not in kwargs:
        # initialize F
        n_clusters = kwargs['n_clusters']
        F = kmeans_initialization(X, n_clusters)
    else:
        F = kwargs['F0']
    if 'verbose' not in kwargs:
        verbose = False
    else:
        verbose = kwargs['verbose']
    
    n_samples, n_features = X.shape

    # initialize D as identity matrix
    D = np.identity(n_features)
    I = np.identity(n_samples)

    # build laplacian matrix
    L = np.array(W.sum(1))[:, 0] - W

    max_iter = 1000
    obj = np.zeros(max_iter)
    for iter_step in range(max_iter):
        # update W
        T = np.linalg.inv(np.dot(X.transpose(), X) + beta * D + 1e-6*np.eye(n_features))
        W = np.dot(np.dot(T, X.transpose()), F)
        # update D
        temp = np.sqrt((W*W).sum(1))
        temp[temp < 1e-16] = 1e-16
        temp = 0.5 / temp
        D = np.diag(temp)
        # update M
        M = L + alpha * (I - np.dot(np.dot(X, T), X.transpose()))
        M = (M + M.transpose())/2
        # update F
        denominator = np.dot(M, F) + gamma*np.dot(np.dot(F, F.transpose()), F)
        temp = np.divide(gamma*F, denominator)
        F = F*np.array(temp)
        temp = np.diag(np.sqrt(np.diag(1 / (np.dot(F.transpose(), F) + 1e-16))))
        F = np.dot(F, temp)

        # calculate objective function
        obj[iter_step] = np.trace(np.dot(np.dot(F.transpose(), M), F)) + gamma/4*np.linalg.norm(np.dot(F.transpose(), F)-np.identity(n_clusters), 'fro')
        if verbose:
            print('obj at iter {0}: {1}'.format(iter_step+1, obj[iter_step]))

        if iter_step >= 1 and math.fabs(obj[iter_step] - obj[iter_step-1]) < 1e-3:
            break
    return W


def kmeans_initialization(X, n_clusters):
    """
    This function uses kmeans to initialize the pseudo label

    Input
    -----
    X: {numpy array}, shape (n_samples, n_features)
        input data
    n_clusters: {int}
        number of clusters

    Output
    ------
    Y: {numpy array}, shape (n_samples, n_clusters)
        pseudo label matrix
    """

    n_samples, n_features = X.shape
    kmeans = sklearn.cluster.KMeans(n_clusters=n_clusters, init='k-means++', n_init=10, max_iter=300,
                                    tol=0.0001, precompute_distances=True, verbose=0,
                                    random_state=4, copy_x=True, n_jobs=1)
    kmeans.fit(X)
    labels = kmeans.labels_
    Y = np.zeros((n_samples, n_clusters))
    for row in range(0, n_samples):
        Y[row, labels[row]] = 1
    T = np.dot(Y.transpose(), Y)
    F = np.dot(Y, np.sqrt(np.linalg.inv(T)))
    F = F + 0.02*np.ones((n_samples, n_clusters))
    return F


def calculate_obj(X, W, F, L, alpha, beta):
    """
    This function calculates the objective function of NDFS
    """
    # Tr(F^T L F)
    T1 = np.trace(np.dot(np.dot(F.transpose(), L), F))
    T2 = np.linalg.norm(np.dot(X, W) - F, 'fro')
    T3 = (np.sqrt((W*W).sum(1))).sum()
    obj = T1 + alpha*(T2 + beta*T3)
    return obj


# Get marker genes from in situ data
insitu_df = pd.read_csv('bdtnp.txt',sep='\t')
marker_genes = list(insitu_df.columns)

# Get expression of marker genes for cells
cell_df = pd.read_csv('dge_normalized.txt',sep='\t')
all_genes = list(cell_df.index)  
cell_df = cell_df.T
cell_df.columns = all_genes
cell_df = cell_df[marker_genes]

# rank features using NDFS feature selection method
with open('fs_ndfs_40.txt', 'w') as f:
    # construct affinity matrix
    kwargs = {"metric": 'euclidean', "neighborMode": "knn", "weightMode": 'heat_kernel', "k": 5, 't': 1}
    W = construct_W(cell_df.values, **kwargs)
    # obtain the feature weight matrix
    score = ndfs(cell_df.values, W=W, n_clusters=5)#len(set([int(e) for e in open('data/cluster_label.txt')]))
    # sort the feature scores in an ascending order according to the feature scores
    idx = feature_ranking(score)
    ls = [marker_genes[e] for e in idx][:40]
    f.write('\n'.join(ls))

### Location prediction for cells

In [3]:
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
import scipy

# this function returns MCC score for true_labels and pred_labels vectors
def get_mcc(true_labels, pred_labels):
    TP = np.sum(np.logical_and(pred_labels == 1, true_labels == 1))
    TN = np.sum(np.logical_and(pred_labels == 0, true_labels == 0))
    FP = np.sum(np.logical_and(pred_labels == 1, true_labels == 0))
    FN = np.sum(np.logical_and(pred_labels == 0, true_labels == 1))
    mcc = (TP * TN) - (FP * FN)
    denom = np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    if denom==0:
        return 0
    return mcc / denom

# turn the probability to the clusters of the cell to the weighted coordinates of the cluster centers
def get_xyz(x):
    ret = []
    for i in range(len(x)):
        v = np.average(cluster_centers, axis=0, weights=x[i,:])
        ret += [ v ]
    return np.array(ret)

# in-situ binary expression
insitu_bin = pd.read_csv('binarized_bdtnp.csv')
# in-situ coordinates
insitu_coords = pd.read_csv('geometry.txt',sep=' ')

# cell binary expression
cell_bin = pd.read_csv('dge_binarized_distMap.csv')
all_genes = list(cell_bin.index)  
cell_bin = cell_bin.T
cell_bin.columns = all_genes

labels = [int(e) for e in open('cluster_label.txt')]
labels = np.array(labels)
cluster_centers = np.load('cluster_centers.npy')


genes = [e.strip() for e in open('fs_ndfs_40.txt') if e.strip()]
clf = KNeighborsClassifier(n_neighbors=10, leaf_size=1, metric='matching')
# learn a multi-class classifier
clf.fit(insitu_bin[genes].values, labels)
# predict probability of cells to the clusters
preds = clf.predict_proba(cell_bin[genes].values)
# turn the probabilities to average weighted coordinates
preds = get_xyz(preds)
# distance to the in situ coordinates of the predicted coordinates for cells
preds_to_insitu_coords = scipy.spatial.distance.cdist(preds,insitu_coords[['xcoord','ycoord','zcoord']].values)

# choose 10 best locations
TOP = 10
# store indices of locations for all cells
indices = []
for i in range(len(preds_to_insitu_coords)):
#     get the index of the location whose the smallest to insitu coords
    best_idx = preds_to_insitu_coords[i].argmin()
#     get coordinates for the location
    coord = insitu_coords[ insitu_coords.index==best_idx ][['xcoord','ycoord','zcoord']].values
#     compared the coord with all the coords    
    dist = scipy.spatial.distance.cdist(coord, insitu_coords[['xcoord','ycoord','zcoord']].values)[0]
#     get the locations with shortest Euclidean distance
    top_idx = dist.argsort()[:TOP]
#     +1 as the location starts from 1
    top_idx = [e+1 for e in top_idx]
    indices += [ top_idx ]


dout = '40genes.csv' 
with open(dout,'w') as file_result:
    tmp = ['']
    for i in range(len(genes)):
        tmp += [ genes[i] ]
        if (i+1)%10==0:
            file_result.write(','.join(tmp) + '\n')
            tmp = ['']
    for i in range(len(indices)):
        tmp = [i+1] + indices[i]
        file_result.write(','.join(map(str,tmp)) + '\n')